In [1]:

%%writefile mapper.py

import sys
import re

reload(sys)
sys.setdefaultencoding('utf-8') # required to convert to unicode

distributed_cache_file = 'stop_words_en.txt'
def read_stop_words(file_path):
    return set(word.strip().lower() for word in open(file_path))
stop_words = read_stop_words(distributed_cache_file)

for line in sys.stdin:
    try:
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        continue
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    for word in words:
        print >> sys.stderr, "reporter:counter:myGroup,Total_words,%d" % 1
        if word in stop_words:
            print >> sys.stderr, "reporter:counter:myGroup,Stop_words,%d" % 1

Overwriting mapper.py


In [2]:

%%writefile myScript.py

import sys
reload(sys)

nums = []
for line in sys.stdin:
    try:
        num = line.strip()
        num = float(num)
#         num = float(num)
    except ValueError as e:
        continue
    
    nums.append(num)
print nums[0]*100.0 / nums[1]

Writing myScript.py


In [3]:
%%bash

OUT_DIR="wordcount_result_"$(date +"%s%6N")
NUM_REDUCERS=0

hdfs dfs -rm -r -skipTrash ${OUT_DIR} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapred.jab.name="Streaming wordCount" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper.py,/datasets/stop_words_en.txt \
    -mapper "python mapper.py" \
    -numReduceTasks 0 \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR} > /dev/null 2> output.log

# print the answer
cat output.log | egrep "*_words" | grep -Eo [0-9]+ | python2 myScript.py

# print log to stderr for grader
cat output.log >&2

38.4403690091


rm: `wordcount_result_1523287356128041': No such file or directory
18/04/09 15:22:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/09 15:22:39 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/04/09 15:22:39 INFO mapred.FileInputFormat: Total input files to process : 1
18/04/09 15:22:40 INFO mapreduce.JobSubmitter: number of splits:2
18/04/09 15:22:40 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1523238091714_0003
18/04/09 15:22:40 INFO impl.YarnClientImpl: Submitted application application_1523238091714_0003
18/04/09 15:22:40 INFO mapreduce.Job: The url to track the job: http://f54c3d191c00:8088/proxy/application_1523238091714_0003/
18/04/09 15:22:40 INFO mapreduce.Job: Running job: job_1523238091714_0003
18/04/09 15:22:46 INFO mapreduce.Job: Job job_1523238091714_0003 running in uber mode : false
18/04/09 15:22:46 INFO mapreduce.Job:  map 0% reduce 0%
18/04/09 15:23:02 INFO mapreduce.Job:  map 53% reduce 0%
18/04/09 15:2